In [1]:
import json
import logging
import networkx as nx
import matplotlib.pyplot as plt
from openai import OpenAI

# 导入自定义模块
import config
import utils

In [ ]:
# 初始化日志
config.setup_logging()
logger = logging.getLogger(__name__)

# 配置 Matplotlib
plt.rcParams['font.sans-serif'] = config.PLOT_SETTINGS.get('font_sans_serif', ['SimHei'])
plt.rcParams['axes.unicode_minus'] = config.PLOT_SETTINGS.get('axes_unicode_minus', False)
prompt_content = utils.load_file_content(config.PROMPT_FILE)
notes_list = utils.load_all_markdown_files(config.DATA_DIR)


2026-02-02 16:04:29,586 - INFO - 在 data 中发现 8 个 Markdown 文件


In [6]:
notes_list[0]

'【文件名: cpplambda.md】\n语法：`[](...){...}`\n- `[]` 捕获列表，为空表示在函数体 `{}` 中，不使用外部变量\n\t- 可选项\n\t\t`[=]` 按值捕获外部变量，即在 `{}` 中用到外部变量\n\t\t`[&]` 按引用捕获外部变量\n- `()` 传递参数，相当于普通函数的外部传参\n- `{}` 函数体，真正实现函数逻辑的部分\n- \n和普通函数的区别：\n普通函数：\n```cpp\nint normalfunction(int argv); // 声明，一般放在main函数之前，用来告诉编译器接下来的mian()函数中可能出现这个函数，不要报错。\nint main(){\n    normalfunction(1);// 调用\n}\nint normalfunction(int argv){ // 定义\n}\n```\n\nlambda 表达式（匿名函数）：\n```cpp\nint main(){\n    bool(* c)(int , int) = [](int a, int b){return a == b;};\n\t// 这段代码中的[](int a, int b){return a == b;}类型是闭包对象，可以隐式转换成函数指针\t\n\t\n\t// 闭包对象是什么？\n\t// 更方便的写法：\n\tauto c = [](int a , int b){return a == b;};\n\t// 这里的c是什么类型？闭包类型\n\tbool d  = c(1, 2);\n\t\n}\n```\n\n具体实现细节\n```cpp\n// 这行语句运行后究竟发生了什么\nauto c = [](int a, int b){return a == b;};\n// \n// 1. 执行lambda 表达式,构建出一个唯一的匿名类类型\n    class eqClosure{\n\t    // private: \n\t        // 如果有捕获外部对象，会在这里产生数据成员\n\t\t\t\n        // 这个类重载了()运算符\n\t\tpublic:\n\t\tbool operator()(int a, int b) const {return a== b;

In [ ]:
client = OpenAI(api_key=config.API_KEY, base_url=config.BASE_URL)
prompt = prompt_content.replace("CONTENT_PLACEHOLDER", notes_list[0])
logger.info(f"{prompt[:50]}... ... {prompt[-50:]}")

2026-02-02 16:07:58,266 - INFO - 你是一个智能笔记分析助手。请将用户的笔记转化为【图谱结构数据】。
【规则】：
1. 忽略情绪化词语（如"哎呀"、"好烦"）。
2. 请提取以下两类信息，并以 JSON 格式输出：
2.1. "triplets": [列表]，包含简单的实体关系。
    示例输出：
    {
        "triplets": [
            {"head": "ConnectionTimeout", "relation": "起因", "tail": "代理配置错误"},
            {"head": "计算机网络课设", "relation": "包含", "tail": "界面要求"}
        ]
    }
2.2. "chunks": [列表]，包含复杂的描述、步骤或要求。
    示例输出：
    {
        "chunks": [
        {"subject": "ConnectionTimeout",
        "predicate": "排查过程",
        "content": "首先检查网络配置，发现代理设为 127.0.0.1:7890 但 Clash 未启动。通过关闭环境变量中的 HTTP_PROXY 解决。"}
        ]
    }
3. 实体尽量标准化（例如 "Docker容器" 统一为 "Docker"）。
4. 如果没有明确关系，返回空列表 []。
【待处理笔记】：
【文件名: cpplambda.md】
语法：`[](...){...}`
- `[]` 捕获列表，为空表示在函数体 `{}` 中，不使用外部变量
	- 可选项
		`[=]` 按值捕获外部变量，即在 `{}` 中用到外部变量
		`[&]` 按引用捕获外部变量
- `()` 传递参数，相当于普通函数的外部传参
- `{}` 函数体，真正实现函数逻辑的部分
- 
和普通函数的区别：
普通函数：
```cpp
int normalfunction(int argv); // 声明，一般放在main函数之前，用来告诉编译器接下来的mian()函数中可能出现这个函数，不要报错。
int main(){
    normalfunction(1);//

In [8]:
response = client.chat.completions.create(
    model=config.MODEL_NAME,
    messages=[{"role": "user", "content": prompt}],
    temperature=config.TEMPERATURE
)
content = response.choices[0].message.content.strip()
logger.info(content)

2026-02-02 16:10:37,773 - INFO - HTTP Request: POST https://dashscope.aliyuncs.com/compatible-mode/v1/chat/completions "HTTP/1.1 200 OK"
2026-02-02 16:10:37,785 - INFO - {
    "triplets": [
        {"head": "lambda表达式", "relation": "包含", "tail": "捕获列表"},
        {"head": "lambda表达式", "relation": "包含", "tail": "参数列表"},
        {"head": "lambda表达式", "relation": "包含", "tail": "函数体"},
        {"head": "普通函数", "relation": "需要", "tail": "声明"},
        {"head": "普通函数", "relation": "需要", "tail": "定义"},
        {"head": "lambda表达式", "relation": "可以转换为", "tail": "函数指针"},
        {"head": "闭包对象", "relation": "是", "tail": "匿名类类型"}
    ],
    "chunks": [
        {
            "subject": "lambda表达式的具体实现",
            "predicate": "过程",
            "content": "1. 执行lambda 表达式,构建出一个唯一的匿名类类型，该类重载了()运算符。2. 生成一个临时变量，相当于创建了一个类的实例。3. 将这个临时变量赋值给c，c这时是lambda表达式生成的匿名类的实例。4. 无捕获的lambda表达式可以转换为函数指针。"
        }
    ]
}
